In [57]:
""" This script tries only using the TuLiP toolbox to solve a multirobot planning execution (MrPE) problem.
A predefined paths are given and a control policy is generated by this script
"""

# Import the packages that we need
from __future__ import print_function
import numpy as np
from tulip import spec
from tulip.transys import machines
from tulip import synth
import gr1_fragment

In [58]:
# Define the parameters & desired trajectories
num_robots = 1  # must be > 1
num_cells = 3
init = np.array([1])
dest = np.array([3])

In [59]:
# Define the env_vars based on the params % trajectories
env_vars = set()
env_vars_list = []
for r in range(1, num_robots + 1):
    env_vars_list.append([])
    for p in range(1, num_cells + 1):
        new_env_var = 's' + str(r) + 'a' + str(p)
        env_vars |= {new_env_var}
        env_vars_list[r - 1].append(new_env_var)

In [60]:
print(env_vars)

{'s1a2', 's1a3', 's1a1'}


In [61]:
print(env_vars_list)

[['s1a1', 's1a2', 's1a3']]


In [62]:
# Define the initial conditions
env_init = set()
for r in range(num_robots):
    env_init |= {env_vars_list[r][init[r] - 1]}

In [63]:
print(env_init)

{'s1a1'}


In [64]:
env_safe = set()
env_prog = set()
# Encode the constraint that each robot can only appear in one position
env_unique = set()
for r in range(1, num_robots + 1):
    for i in range(len(env_vars_list[r - 1])):
        new_env_unique = '( ' + env_vars_list[r - 1][i] + ' ) <-> ( '
        for j in range(len(env_vars_list[r - 1])):
            if j != i:
                new_env_unique += '!' + env_vars_list[r - 1][j] + ' && '
        new_env_unique = new_env_unique[:-3] + ')'
        env_unique |= {new_env_unique}
env_safe |= env_unique

In [65]:
print(env_unique)

{'( s1a1 ) <-> ( !s1a2 && !s1a3 )', '( s1a3 ) <-> ( !s1a1 && !s1a2 )', '( s1a2 ) <-> ( !s1a1 && !s1a3 )'}


In [66]:
# Encode the transition system relations
# transition = np.array([[0, 2, 7, 5], [1, 3, 0, 0], [2, 4, 0, 6], [3, 0, 0, 8], [0, 0, 1, 0], [0, 0, 3, 0], [0, 0, 0, 1], [0, 0, 4, 0]])
# transition = np.array([[0, 2, 0, 0], [1, 0, 0, 0]])
transition = np.array([[0, 2, 0, 0], [1, 3, 0, 0], [2, 0, 0, 0]])
actions = ['up', 'down', 'left', 'right']
env_trans = set()
sys_trans = set()
# env_trans |= {'( s1a1 ) -> ( stop1 || down1 )'}
# env_trans |= {'( s1a2 ) -> ( stop1 || up1 )'}
for i in range(1, num_cells + 1):
    for r in range(num_robots):
        for d in range(4):
            if transition[i - 1][d] > 0:
                env_trans |= {'( ' + env_vars_list[r][i - 1] + ' && ' + actions[d] + str(r + 1) + ' ) -> X( ' +
                            env_vars_list[r][i - 1] + ' || ' + env_vars_list[r][transition[i - 1][d] - 1] + ' )'}
                # env_trans |= {'( ' + env_vars_list[r][i - 1] + ' && ' + actions[d] + str(r + 1) + ' ) -> X( ' +
                            # env_vars_list[r][transition[i - 1][d] - 1] + ' )'}
            else:
                sys_trans |= {'( ' + env_vars_list[r][i - 1] + ' ) -> ( !' + actions[d] + str(r + 1) + ' )'}
                # env_trans |= {'( ' + env_vars_list[r][i - 1] + ' && ' + actions[d] + str(r + 1) + ' ) -> X( ' +
                  #    env_vars_list[r][i - 1] + ' )'}
        env_trans |= {'( ' + env_vars_list[r][i - 1] + ' && ' + 'stop' + str(r + 1) + ' ) -> X( ' +
                      env_vars_list[r][i - 1] + ' )'}

env_safe |= env_trans

In [67]:
print(env_trans)

{'( s1a1 && down1 ) -> X( s1a1 || s1a2 )', '( s1a2 && down1 ) -> X( s1a2 || s1a3 )', '( s1a3 && stop1 ) -> X( s1a3 )', '( s1a1 && stop1 ) -> X( s1a1 )', '( s1a2 && up1 ) -> X( s1a2 || s1a1 )', '( s1a2 && stop1 ) -> X( s1a2 )', '( s1a3 && up1 ) -> X( s1a3 || s1a2 )'}


In [68]:
print(len(env_trans))

7


In [69]:
# Encode the constraint that every robot should stay after reaching its destination
env_dest_stop = set()
sys_dest = set()
for r in range(num_robots):
    env_dest_stop |= {'( ' + env_vars_list[r][dest[r] - 1] + ' ) -> X( ' + env_vars_list[r][dest[r] - 1] + ' )'}
    sys_dest |= {env_vars_list[r][dest[r] - 1]}
# env_safe |= env_dest_stop

In [70]:
print(env_dest_stop)

{'( s1a3 ) -> X( s1a3 )'}


In [71]:
print(sys_dest)

{'s1a3'}


In [72]:
# Encode the constraint that each robot should either not go or eventually not in the current cell
env_go = set()
for r in range(1, num_robots + 1):
    for i in range(num_cells):
        if i != dest[r - 1] - 1:
            env_go |= {'( !' + env_vars_list[r - 1][i] + ' || ' + 'stop' + str(r) + ' )'}
env_prog |= env_go

In [73]:
print(env_go)

{'( !s1a1 || stop1 )', '( !s1a2 || stop1 )'}


In [74]:
# Define sys_vars
sys_vars = set()
for r in range(1, num_robots + 1):
    sys_vars |= {'stop' + str(r)}
    sys_vars |= {'left' + str(r)}
    sys_vars |= {'right' + str(r)}
    sys_vars |= {'up' + str(r)}
    sys_vars |= {'down' + str(r)}

In [75]:
print(sys_vars)

{'right1', 'up1', 'stop1', 'down1', 'left1'}


In [76]:
# Define initial command. Assume nothing.
sys_init = set()

sys_safe = set()
sys_prog = set()
sys_prog |= sys_dest
sys_safe |= env_dest_stop
sys_safe |= sys_trans
# Encode the constraint that each time one action must be selected
env_act = set()
for r in range(1, num_robots + 1):
    env_act |= {'( left' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    env_act |= {'( right' + str(r) + ' ) <-> ( ' + '!left' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    env_act |= {'( up' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!left' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    env_act |= {'( down' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!left' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    env_act |= {'( stop' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!left' + str(r) + ' )'}
    
sys_safe |= env_act
print(env_act)

{'( stop1 ) <-> ( !right1 && !up1 && !down1 && !left1 )', '( left1 ) <-> ( !right1 && !up1 && !down1 && !stop1 )', '( down1 ) <-> ( !right1 && !up1 && !left1 && !stop1 )', '( right1 ) <-> ( !left1 && !up1 && !down1 && !stop1 )', '( up1 ) <-> ( !right1 && !left1 && !down1 && !stop1 )'}


In [77]:
# Encode the constraint that if ending point is reached, not go. And if ending point is not reached, always exist some
# robot that goes
sys_stop = set()
new_sys_nonstop = '( '
for r in range(1, num_robots + 1):
    sys_stop |= {'( ' + env_vars_list[r - 1][dest[r - 1] - 1] + ' ) -> ( ' + 'stop' + str(r) + ' )'}
    new_sys_nonstop += '( !' + env_vars_list[r - 1][dest[r - 1] - 1] + ' ) || '
new_sys_nonstop = new_sys_nonstop[:-3] + ') -> ( '
for r in range(1, num_robots + 1):
    new_sys_nonstop += '!stop' + str(r) + ' || '
new_sys_nonstop = new_sys_nonstop[:-3] + ')'
sys_stop |= {new_sys_nonstop}
sys_safe |= sys_stop

In [78]:
print(sys_stop)

{'( ( !s1a3 ) ) -> ( !stop1 )', '( s1a3 ) -> ( stop1 )'}


In [79]:
# Encode the collision avoidance constraints
# No need if only one robot
sys_collision = set()
for i in range(num_cells):
    new_sys_collision = '!( '
    for r in range(num_robots):
        new_sys_collision += env_vars_list[r][i] + ' && '
    new_sys_collision = new_sys_collision[:-3] + ')'
    sys_collision |= {new_sys_collision}
# sys_safe |= sys_collision

In [80]:
print(sys_collision)

{'!( s1a1 )', '!( s1a3 )', '!( s1a2 )'}


In [81]:
# Create a GR(1) specification
specs = spec.GRSpec(env_vars, sys_vars, env_init, sys_init,
                    env_safe, sys_safe, env_prog, sys_prog)
specs.qinit = '\A \E'  # Moore initial condition synthesized too
specs.moore = False
specs.plus_one = False

In [82]:
print(specs)

((s1a1) && [](( s1a3 ) <-> ( !s1a1 && !s1a2 )) && [](( s1a2 && up1 ) -> X( s1a2 || s1a1 )) && [](( s1a2 && stop1 ) -> X( s1a2 )) && [](( s1a3 && up1 ) -> X( s1a3 || s1a2 )) && [](( s1a1 && down1 ) -> X( s1a1 || s1a2 )) && [](( s1a2 && down1 ) -> X( s1a2 || s1a3 )) && [](( s1a3 && stop1 ) -> X( s1a3 )) && [](( s1a2 ) <-> ( !s1a1 && !s1a3 )) && [](( s1a1 ) <-> ( !s1a2 && !s1a3 )) && [](( s1a1 && stop1 ) -> X( s1a1 )) && []<>(( !s1a1 || stop1 )) && []<>(( !s1a2 || stop1 ))) -> ([](( s1a1 ) -> ( !right1 )) && [](( s1a1 ) -> ( !up1 )) && [](( ( !s1a3 ) ) -> ( !stop1 )) && [](( left1 ) <-> ( !right1 && !up1 && !down1 && !stop1 )) && [](( right1 ) <-> ( !left1 && !up1 && !down1 && !stop1 )) && [](( s1a3 ) -> ( stop1 )) && [](( s1a1 ) -> ( !left1 )) && [](( s1a3 ) -> ( !left1 )) && [](( s1a3 ) -> ( !down1 )) && [](( s1a2 ) -> ( !left1 )) && [](( up1 ) <-> ( !right1 && !left1 && !down1 && !stop1 )) && [](( s1a2 ) -> ( !right1 )) && [](( down1 ) <-> ( !right1 && !up1 && !left1 && !stop1 )) && []

In [83]:
print('Start synthesis')
ctrl = synth.synthesize(specs)
print('End synthesis')
assert ctrl is not None, 'unrealizable'

Start synthesis
removed 4 nodes from 8 total
End synthesis


In [84]:
machines.random_run(ctrl, N=20)

move from
	 state: Sinit
	 with input:{'s1a2': False, 's1a3': False, 's1a1': True}
	 to state: 0
	 reacting by producing output: {'right1': False, 'up1': False, 'stop1': False, 'down1': True, 'left1': False}
move from
	 state: 0
	 with input:{'s1a2': False, 's1a3': False, 's1a1': True}
	 to state: 0
	 reacting by producing output: {'right1': False, 'up1': False, 'stop1': False, 'down1': True, 'left1': False}
move from
	 state: 0
	 with input:{'s1a2': False, 's1a3': False, 's1a1': True}
	 to state: 0
	 reacting by producing output: {'right1': False, 'up1': False, 'stop1': False, 'down1': True, 'left1': False}
move from
	 state: 0
	 with input:{'s1a2': False, 's1a3': False, 's1a1': True}
	 to state: 0
	 reacting by producing output: {'right1': False, 'up1': False, 'stop1': False, 'down1': True, 'left1': False}
move from
	 state: 0
	 with input:{'s1a2': True, 's1a3': False, 's1a1': False}
	 to state: 4
	 reacting by producing output: {'right1': False, 'up1': False, 'stop1': False, 'down1'

([0, 0, 0, 0, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
 {'right1': [False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False],
  'up1': [False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False],
  'stop1': [False,
   False,
   False,
   False,
   False,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True],
  'down1': [True,
   True,
   True,
   True,
   True,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False],
  'left1': [False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,

In [57]:
print(len(env_unique))

16
